In [20]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/Users/moneysniper/Documents/NBA_analysis_project/gamelogs/nba_games_cleansed.csv')
df = df.iloc[[i for i in range(len(df)) if i % 4 in (0, 3)]]

df

# List of variables for the pivot table
variables = ['diff_orb', 'diff_tov', 'diff_3p%', 'diff_2p%', 'diff_ft%', 'diff_fta', 'diff_3pa']

# Add a 'Location' column for home/away
df['Location'] = df['home'].apply(lambda x: 'Home' if x == 1 else 'Away')

# Add a 'Year' column
df['Year'] = pd.to_datetime(df['date']).dt.year

# Function to create a pivot for a specific variable
def create_pivot(variable):
    pivot = df.pivot_table(
        index=['Location'],
        columns='Year',
        values=variable,
        aggfunc='mean'
    )
    pivot['Total'] = pivot.mean(axis=1)
    pivot.loc['Difference'] = pivot.loc['Home'] - pivot.loc['Away']
    return pivot

# Generate and display the pivot tables for all variables
pivot_tables = {var: create_pivot(var) for var in variables}

# Display the pivot tables
for var, pivot in pivot_tables.items():
    print(f"\nPivot Table for {var}:\n")
    print(pivot)


Pivot Table for diff_orb:

Year            2015      2016      2017      2018      2019      2020  \
Location                                                                 
Away       -0.229249 -0.267164 -0.151976 -0.518038  0.034268 -0.059524   
Home        0.228814  0.177979  0.052250  0.260450  0.614400  0.439891   
Difference  0.458063  0.445143  0.204225  0.778488  0.580132  0.499415   

Year            2021      2022      2023      2024     Total  
Location                                                      
Away       -0.209476 -0.134128  0.154093 -0.474201 -0.185540  
Home        0.026538  0.143070  0.011076  0.166271  0.212074  
Difference  0.236014  0.277198 -0.143017  0.640472  0.397613  

Pivot Table for diff_tov:

Year            2015     2016      2017      2018      2019      2020  \
Location                                                                
Away        0.648221  0.10597  0.057751  0.066378  0.294393  0.080357   
Home       -0.216102 -0.47813 -0.402032

In [23]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/Users/moneysniper/Documents/NBA_analysis_project/gamelogs/nba_games_cleansed.csv')

# Keep only rows where index % 4 == 0 or index % 4 == 3
df = df.iloc[[i for i in range(len(df)) if i % 4 in (0, 3)]]

# Ensure 'won' column exists
if 'won' not in df.columns:
    raise KeyError("Column 'won' not found in the dataset. Please check column names.")

# Add a 'Year' column
df['Year'] = pd.to_datetime(df['date']).dt.year

# List of variables for the pivot table
variables = ['diff_orb', 'diff_tov', 'diff_3p%', 'diff_2p%', 'diff_ft%', 'diff_fta', 'diff_3pa']

# Create a MultiIndex DataFrame to store all results in one table
pivot_data = []

for variable in variables:
    pivot = df.pivot_table(
        index=['won'],  # Group by win/loss
        columns='Year',
        values=variable,
        aggfunc='mean'
    )
    pivot['Total'] = pivot.mean(axis=1)  # Calculate overall mean per group
    pivot.rename(index={0: 'Loss', 1: 'Win'}, inplace=True)  # Rename index for clarity
    pivot.loc['Difference'] = pivot.loc['Win'] - pivot.loc['Loss']  # Compute win-loss difference
    pivot['Variable'] = variable  # Add variable column for stacking
    pivot.reset_index(inplace=True)  # Reset index to flatten table
    pivot_data.append(pivot)

# Combine all pivots into a single DataFrame
final_pivot = pd.concat(pivot_data, ignore_index=True)

# Rearrange columns
cols = ['Variable', 'won'] + [col for col in final_pivot.columns if col not in ['Variable', 'won']]
final_pivot = final_pivot[cols]

final_pivot

Year,Variable,won,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,Total
0,diff_orb,Loss,0.141079,0.288520,0.232295,-0.162080,0.196697,0.363636,0.103627,0.103393,0.116751,0.141439,0.152536
1,diff_orb,Win,-0.153226,-0.375559,-0.355694,-0.137670,0.457571,0.047091,-0.263097,-0.080221,0.051205,-0.423529,-0.123313
2,diff_orb,Difference,-0.294305,-0.664079,-0.587989,0.024409,0.260874,-0.316545,-0.366724,-0.183614,-0.065546,-0.564969,-0.275849
3,diff_tov,Loss,1.149378,0.859517,0.541076,0.975535,0.687688,1.064516,0.770725,0.891761,0.868020,1.054591,0.886281
4,diff_tov,Win,-0.661290,-1.214605,-0.968799,-0.968230,-0.430948,-0.590028,-0.532014,-0.793914,-1.075301,-0.894118,-0.812925
5,diff_tov,Difference,-1.810668,-2.074122,-1.509875,-1.943765,-1.118636,-1.654544,-1.302739,-1.685675,-1.943322,-1.948708,-1.699205
6,diff_3p%,Loss,-6.547303,-7.070997,-6.786119,-5.117737,-6.154354,-6.504985,-6.612694,-5.499515,-5.297124,-6.232258,-6.182309
7,diff_3p%,Win,6.567742,6.112370,5.961778,6.452950,5.469052,5.811080,6.370664,6.299170,6.299548,5.937647,6.128200
8,diff_3p%,Difference,13.115045,13.183367,12.747897,11.570687,11.623406,12.316066,12.983358,11.798685,11.596672,12.169905,12.310509
9,diff_2p%,Loss,-4.360792,-4.567909,-4.749004,-4.458929,-4.252868,-4.353834,-4.307983,-4.286069,-4.586398,-5.506300,-4.543008
